In [1]:
import os
import re
import json
import traceback
import numpy as np
import huggingface_hub

from tqdm import tqdm
from openai import OpenAI

from kiwipiepy import Kiwi ## 한글 형태소 분석기

from langchain.schema import Document
from langchain.vectorstores import FAISS  ## 벡터 DB
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import JSONLoader
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

from dotenv import load_dotenv
load_dotenv("../keys.env")

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

hf_token = os.getenv("HF_TOKEN")
huggingface_hub.login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/pervinco/.cache/huggingface/token
Login successful


In [2]:
# 문서 로드 함수
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

# 문서 로드
raw_documents = load_jsonl('../dataset/documents.jsonl')

# 문서에서 docid를 포함한 Document 리스트 생성
documents = []
for doc in raw_documents:
    doc_id = doc['docid']  # JSONL 파일에서 docid 추출
    content = doc['content']  # 문서 내용 추출
    documents.append(Document(page_content=content, metadata={"docid": doc_id}))

In [3]:
# KIWI로 토큰화 함수 정의
kiwi = Kiwi()
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

In [4]:
embedding_model = OpenAIEmbeddings()

# BM25와 FAISS 기반 검색기 생성
bm25 = BM25Retriever.from_documents(documents)
kiwi_bm25 = BM25Retriever.from_documents(documents, preprocess_func=kiwi_tokenize)

# FAISS 벡터 DB 생성 및 메타데이터와 함께 저장
faiss = FAISS.from_documents(documents, embedding_model).as_retriever()

bm25_faiss_73 = EnsembleRetriever(
    retrievers=[bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.7, 0.3],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)
bm25_faiss_37 = EnsembleRetriever(
    retrievers=[bm25, faiss],
    weights=[0.3, 0.7],
    search_type="mmr",
)
kiwibm25_faiss_73 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],
    weights=[0.7, 0.3],
    search_type="mmr",
)
kiwibm25_faiss_37 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],
    weights=[0.3, 0.7],
    search_type="mmr",
)

In [5]:
retrievers = {
    "bm25": bm25,
    "kiwi_bm25": kiwi_bm25,
    "faiss": faiss,
    "bm25_faiss_73": bm25_faiss_73,
    "bm25_faiss_37": bm25_faiss_37,
    "kiwi_bm25_faiss_73": kiwibm25_faiss_73,
    "kiwi_bm25_faiss_37": kiwibm25_faiss_37,
}

In [6]:
# 쿼리 임베딩 벡터 생성 함수
def get_query_embedding(query, embedding_model):
    query_embedding = embedding_model.embed_query(query)
    return np.array(query_embedding)  # 리스트를 넘파이 배열로 변환

# L2 Distance 계산 함수
def calculate_l2_distance(vec1, vec2):
    return np.linalg.norm(vec1 - vec2)

# 검색 결과에서 L2 Distance 계산 후 출력하는 함수
def print_search_results_with_l2(retrievers, query, embedding_model):
    print(f"Query : {query}")
    
    # 쿼리 임베딩 벡터 생성
    query_embedding = get_query_embedding(query, embedding_model)
    
    for name, retriever in retrievers.items():
        try:
            results = retriever.invoke(query)  # LangChain 최신 버전 메서드 사용
            
            print(f"{name} 결과:")
            for result in results:
                # 검색된 문서의 임베딩 벡터 가져오기
                doc_embedding = embedding_model.embed_documents([result.page_content])[0]
                doc_embedding = np.array(doc_embedding)  # 리스트를 넘파이 배열로 변환
                
                # 쿼리 임베딩과 문서 임베딩 간의 L2 Distance 계산
                l2_distance = calculate_l2_distance(query_embedding, doc_embedding)
                
                print(f"문서 ID: {result.metadata['docid']} L2 Distance: {l2_distance} 내용: {result.page_content}")
            print("===" * 20)
        except Exception as e:
            print(f"{name} 검색 중 오류 발생: {e}")
            print("===" * 20)

In [7]:
print_search_results_with_l2(retrievers, "ktx의 평균 속도는 얼마인가?", embedding_model)

Query : ktx의 평균 속도는 얼마인가?
bm25 결과:
문서 ID: e8d07278-c590-43e2-b99a-47d5adaf123d L2 Distance: 0.5719227922623585 내용: 아르투로는 3,000미터 경주에서 10분이라는 기록으로 결승점에 도달했습니다. 이를 통해 우리는 아르투로의 평균 속도를 계산할 수 있습니다. 평균 속도는 이동한 거리를 이동에 걸린 시간으로 나눈 값입니다. 따라서, 아르투로의 평균 속도는 3,000미터를 10분으로 나눈 값인 300미터/분입니다. 이를 미터/초로 변환하면 5 m/s가 됩니다. 따라서, 아르투로의 평균 속도는 5 m/s입니다.
문서 ID: 8ca2ebf5-9f0d-4d2f-bfef-92b44c920d2e L2 Distance: 0.5882072158385281 내용: 고무 오리 경주에서 우승한 오리는 10분 동안 300미터를 이동했습니다. 이를 통해 우승한 오리의 평균 속도를 계산할 수 있습니다. 평균 속도는 이동한 거리를 이동에 소요된 시간으로 나눈 값입니다. 따라서, 이 경우 오리의 평균 속도는 300미터를 10분으로 나눈 값인 30미터/분입니다. 그러나, 문제에서 속도를 미터/초로 표현하라고 하였으므로, 분 단위를 초 단위로 변환해야 합니다. 1분은 60초이므로, 30미터/분은 0.5미터/초입니다. 따라서, 고무 오리 경주에서 우승한 오리의 평균 속도는 0.5미터/초입니다.
문서 ID: 525f7b96-8422-4e57-b54b-32aff1d42aed L2 Distance: 0.5849022861254208 내용: 학생들은 학교 운동장에서 거북이를 관찰하였습니다. 그들은 거북이가 30분 동안 이동한 거리를 측정하였습니다. 거북이가 이 시간 동안 40미터를 걸었다면, 평균 속도는 얼마였을까요? 이 문제를 풀기 위해서는 거리를 시간으로 나누어야 합니다. 따라서, 거북이의 평균 속도는 40미터를 30분으로 나눈 값인 1.33m/min입니다. 그러나, 답안에서는 속도를 시간당 단위로 표기하였습니다

In [8]:
# 쿼리 임베딩 벡터 생성 함수
def get_query_embedding(query, embedding_model):
    query_embedding = embedding_model.embed_query(query)
    return np.array(query_embedding)  # 리스트를 넘파이 배열로 변환

# 코사인 유사도 계산 함수
def calculate_cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

# 검색 결과에서 코사인 유사도 계산 후 출력하는 함수
def print_search_results_with_cosine(retrievers, query, embedding_model):
    print(f"Query : {query}")
    
    # 쿼리 임베딩 벡터 생성
    query_embedding = get_query_embedding(query, embedding_model)
    
    for name, retriever in retrievers.items():
        try:
            results = retriever.invoke(query)  # 최신 LangChain 메서드 사용
            
            print(f"{name} 결과:")
            for result in results:
                # 검색된 문서의 임베딩 벡터 가져오기
                doc_embedding = embedding_model.embed_documents([result.page_content])[0]
                doc_embedding = np.array(doc_embedding)  # 리스트를 넘파이 배열로 변환
                
                # 쿼리 임베딩과 문서 임베딩 간의 코사인 유사도 계산
                cosine_similarity = calculate_cosine_similarity(query_embedding, doc_embedding)
                
                print(f"문서 ID: {result.metadata['docid']} 코사인 유사도: {cosine_similarity} 내용: {result.page_content}")
            print("===" * 20)
        except Exception as e:
            print(f"{name} 검색 중 오류 발생: {e}")
            print("===" * 20)

In [9]:
print_search_results_with_cosine(retrievers, "ktx의 평균 속도는 얼마인가?", embedding_model)

Query : ktx의 평균 속도는 얼마인가?
bm25 결과:
문서 ID: e8d07278-c590-43e2-b99a-47d5adaf123d 코사인 유사도: 0.8364419934303825 내용: 아르투로는 3,000미터 경주에서 10분이라는 기록으로 결승점에 도달했습니다. 이를 통해 우리는 아르투로의 평균 속도를 계산할 수 있습니다. 평균 속도는 이동한 거리를 이동에 걸린 시간으로 나눈 값입니다. 따라서, 아르투로의 평균 속도는 3,000미터를 10분으로 나눈 값인 300미터/분입니다. 이를 미터/초로 변환하면 5 m/s가 됩니다. 따라서, 아르투로의 평균 속도는 5 m/s입니다.
문서 ID: 8ca2ebf5-9f0d-4d2f-bfef-92b44c920d2e 코사인 유사도: 0.8269712195818806 내용: 고무 오리 경주에서 우승한 오리는 10분 동안 300미터를 이동했습니다. 이를 통해 우승한 오리의 평균 속도를 계산할 수 있습니다. 평균 속도는 이동한 거리를 이동에 소요된 시간으로 나눈 값입니다. 따라서, 이 경우 오리의 평균 속도는 300미터를 10분으로 나눈 값인 30미터/분입니다. 그러나, 문제에서 속도를 미터/초로 표현하라고 하였으므로, 분 단위를 초 단위로 변환해야 합니다. 1분은 60초이므로, 30미터/분은 0.5미터/초입니다. 따라서, 고무 오리 경주에서 우승한 오리의 평균 속도는 0.5미터/초입니다.
문서 ID: 525f7b96-8422-4e57-b54b-32aff1d42aed 코사인 유사도: 0.8289181761116802 내용: 학생들은 학교 운동장에서 거북이를 관찰하였습니다. 그들은 거북이가 30분 동안 이동한 거리를 측정하였습니다. 거북이가 이 시간 동안 40미터를 걸었다면, 평균 속도는 얼마였을까요? 이 문제를 풀기 위해서는 거리를 시간으로 나누어야 합니다. 따라서, 거북이의 평균 속도는 40미터를 30분으로 나눈 값인 1.33m/min입니다. 그러나, 답안에서는 속도를 시간당 단위로 표기하였습니다. 따라서, 거북이의 

In [10]:
client = OpenAI()
model = "gpt-4o"

In [11]:
def clean_json_response(response):
    # 코드 블록(예: ```json, ```) 제거
    cleaned_response = re.sub(r'```(?:json)?', '', response).strip()
    return cleaned_response
standalone_content = ("입력된 내용이 한 줄의 문장인지 여러 줄의 대화 내용인지 분류하세요. 반드시 JSON 형식으로 응답하며, 키와 값은 모두 이중 따옴표로 감쌉니다."
                      "- 단일 문장인 경우 : {\"multi_turn\": false, \"query\": \"입력 문장\"} "
                      "- 여러 줄의 대화 내용인 경우 : {\"multi_turn\": true, \"query\": \"대화 내용을 종합하여 만든 새로운 질문\"}")


def create_standalone_query(query):
    # query가 중첩된 리스트 형태일 때 텍스트를 추출
    if isinstance(query, list) and 'content' in query[0]:
        query = query[0]['content'][0]['content']
    
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": standalone_content},
            {"role": "user", "content": query}
        ],
    )
    
    response = completion.choices[0].message.content
    cleaned_response = clean_json_response(response)
    
    try:
        json_response = json.loads(cleaned_response)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON response", "response": response}
    
    return json_response

domain_content = (
    "입력된 텍스트가 상식에 대한 질문인지 아니면 단순한 일상적인 대화인지 구분하세요."
    " - 상식에 대한 질문이란, 사용자가 지식이나 정보를 얻기 위해 하는 질문입니다. 예: '나무의 분류에 대해 조사하는 방법은?', 'Dmitri Ivanovsky가 누구야?', '남녀 관계에서 정서적인 행동이 왜 중요해?'"
    " - 일상적인 대화란, 주로 감정이나 의견을 표현하거나 대화의 흐름을 유지하기 위한 내용입니다. 예: '요새 너무 힘들다.', '니가 대답을 잘해줘서 너무 신나!', '이제 그만 얘기해!', '오늘 너무 즐거웠다!', ''너는 누구니?', '너는 어떤 능력을 가지고 있니?"
    " 반드시 JSON 형식으로 응답하세요. 키와 값은 모두 이중 따옴표로 감싸야 합니다."
    " - 상식에 대한 질문인 경우: {\"out_of_domain\": false, \"query\": \"입력된 쿼리를 그대로 반환\"}"
    " - 일상적인 대화인 경우: {\"out_of_domain\": true, \"query\": \"적절한 응답을 할 수 없습니다.\"}"
)


def domain_check(query):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": domain_content},
            {"role": "user", "content": query}
        ],
    )
    
    response = completion.choices[0].message.content
    cleaned_response = clean_json_response(response)
    
    try:
        json_response = json.loads(cleaned_response)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON response", "response": response}
    
    return json_response

In [12]:
# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
persona_qa = """
## Role: 과학 상식 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
"""

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
persona_function_calling = """
## Role: 과학 상식 전문가

## Instruction
- 사용자가 대화를 통해 과학 지식에 관한 주제로 질문하면 search api를 호출할 수 있어야 한다.
- 과학 상식과 관련되지 않은 나머지 대화 메시지에는 적절한 대답을 생성한다.
"""

# Function calling에 사용할 함수 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "search relevant documents",
            "parameters": {
                "properties": {
                    "standalone_query": {
                        "type": "string",
                        "description": "Final query suitable for use in search from the user messages history."
                    }
                },
                "required": ["standalone_query"],
                "type": "object"
            }
        }
    },
]

In [13]:
def answer_question(messages, embedding_model, retriever):
    # 함수의 출력값 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}
    
    # 쿼리 검사1: 멀티턴 대화인 경우 standalone query를 생성한다.
    result1 = create_standalone_query(messages[0]['content'])
    
    # 쿼리 검사2: 쿼리가 과학 상식과 관련된 것인지 검사한다.
    result2 = domain_check(result1['query'])
    
    if not result2.get('out_of_domain', True):
        query = result2['query']
        response['standalone_query'] = query
        
        search_result = retrievers[retriever].get_relevant_documents(query)
        
        if not search_result:
            response["answer"] = "관련된 문서를 찾을 수 없습니다."
            return response
        
        top3_results = search_result[:3]
        query_embedding = np.array(embedding_model.embed_query(query))
        
        
        retrieved_context = []
        for doc in top3_results:
            # 문서 임베딩 벡터 가져오기
            doc_embedding = np.array(embedding_model.embed_documents([doc.page_content])[0])  # 넘파이 배열로 변환
            
            # 쿼리 임베딩과 문서 임베딩 간의 L2 Distance 계산
            l2_distance = calculate_l2_distance(query_embedding, doc_embedding)
            
            retrieved_context.append(doc.page_content)
            response["topk"].append(doc.metadata.get('docid'))
            response["references"].append({
                "score": l2_distance,  # L2 Distance를 점수로 사용
                "content": doc.page_content
            })
        
        # 검색 결과를 assistant 메시지에 추가 (문자열로 변환)
        content = "\n".join(retrieved_context)
        messages.append({"role": "assistant", "content": content})
        
        # 검색된 문서들을 바탕으로 최종 답변 생성
        msg = [{"role": "system", "content": persona_qa}] + messages
        try:
            qaresult = client.chat.completions.create(
                model=model,
                messages=msg,
                temperature=0,
                seed=1,
                timeout=30
            )
            response["answer"] = qaresult.choices[0].message.content
        
        except Exception as e:
            traceback.print_exc()
            response["answer"] = "답변 생성 중 오류가 발생했습니다."
    
    else:
        response["answer"] = "질문이 과학 상식에 해당하지 않습니다."
    
    return response

In [14]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출후 파일에 저장
def eval_rag(eval_filename, output_filename, embedding_model, retriever):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            print(f"{idx:>04}")
            j = json.loads(line)
            response = answer_question(j["msg"], embedding_model, retriever)
            # print(f'Test {idx:>04}\nQuestion: {j["msg"]}')
            # print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1

In [16]:
retrievers = {
    "bm25": bm25,
    "kiwi_bm25": kiwi_bm25,
    "faiss": faiss,
    "bm25_faiss_73": bm25_faiss_73,
    "bm25_faiss_37": bm25_faiss_37,
    "kiwi_bm25_faiss_73": kiwibm25_faiss_73,
    "kiwi_bm25_faiss_37": kiwibm25_faiss_37,
}

# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("../dataset/eval.jsonl", "../dataset/sample_submission.csv", embedding_model, "kiwi_bm25")

0000
0001
0002
0003
0004
0005
0006
0007
0008
0009
0010
0011
0012
0013
0014
0015
0016
0017
0018
0019
0020
0021
0022
0023
0024
0025
0026
0027
0028
0029
0030
0031
0032
0033
0034
0035
0036
0037
0038
0039
0040
0041
0042
0043
0044
0045
0046
0047
0048
0049
0050
0051
0052
0053
0054
0055
0056
0057
0058
0059
0060
0061
0062
0063
0064
0065
0066
0067
0068
0069
0070
0071
0072
0073
0074
0075
0076
0077
0078
0079
0080
0081
0082
0083
0084
0085
0086
0087
0088
0089
0090
0091
0092
0093
0094
0095
0096
0097
0098
0099
0100
0101
0102
0103
0104
0105
0106
0107
0108
0109
0110
0111
0112
0113
0114
0115
0116
0117
0118
0119
0120
0121
0122
0123
0124
0125
0126
0127
0128
0129
0130
0131
0132
0133
0134
0135
0136
0137
0138
0139
0140
0141
0142
0143
0144
0145
0146
0147
0148
0149
0150
0151
0152
0153
0154
0155
0156
0157
0158
0159
0160
0161
0162
0163
0164
0165
0166
0167
0168
0169
0170
0171
0172
0173
0174
0175
0176
0177
0178
0179
0180
0181
0182
0183
0184
0185
0186
0187
0188
0189
0190
0191
0192
0193
0194
0195
0196
0197
0198
0199
